In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest, f_regression, chi2, SelectPercentile, mutual_info_regression
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.neighbors import LocalOutlierFactor
import math
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score

In [56]:
SEED = 4426008 #np.random.randint(2**32)
np.random.seed(SEED)

In [57]:
just_scoring = False
folder = "drive/MyDrive/AML_Pr_1/featured_reduced/main/"
if just_scoring:
  folder = folder + "just_scoring/"
else:
  folder = folder + "testing/"
raw = folder + "featured_reduced/"
preprocessed = folder + "results/"
super_raw = folder + "featured_reduced2/"

In [58]:
just_scoring = False
folder = "drive/MyDrive/AML_Pr_1/featured_reduced/main/"
if just_scoring:
  folder = folder + "just_scoring/"
else:
  folder = folder + "testing/"
raw = folder + "featured_reduced/"
preprocessed = folder + "results/"
super_raw = folder + "featured_reduced2/"

In [59]:
x_train = pd.read_csv(raw + 'X_train_preprocessed.csv',index_col=['id'])
x_test = pd.read_csv(raw + 'X_test_preprocessed.csv',index_col=['id'])
y_train = pd.read_csv(raw + 'y_train_preprocessed.csv',index_col=['id'])
y_test = pd.read_csv(raw + 'y_test_preprocessed.csv',index_col=['id'])

x_train_etr = pd.read_csv(super_raw + 'X_train_etr_preprocessed.csv',index_col=['id'])
x_test_etr = pd.read_csv(super_raw + 'X_test_etr_preprocessed.csv',index_col=['id'])

In [60]:
TRAIN_RATIO = 0.8

def train_len(data = None):
    if data is None:
        data = x_train
    return math.floor(TRAIN_RATIO*len(data))

def validation_len(data = None):
    if data is None:
        data = x_train
    return len(data) - train_len(data)

def train(data = None):
    if data is None:
        data = x_train
    if isinstance(data, np.ndarray):
        return data[:train_len(data)].copy()
    return data.iloc[:train_len(data)].copy()

def validation(data = None):
    if data is None:
        data = x_train
    if isinstance(data, np.ndarray):
        return data[-validation_len(data):].copy()
    return data.iloc[-validation_len(data):].copy()

def full_X(x_tr = None, x_ts = None):
    if x_tr is None:
        x_tr = x_train
    if x_ts is None:
        x_ts = x_test
    return pd.concat([x_tr, x_ts], join = "inner")

def performance(y_pred):
    if not y_test.empty:
        return r2_score(y_test, y_pred)

In [61]:
x_train_50 = x_train_etr
x_test_50 = x_test_etr

x_train = x_train.drop(x_train_50.columns,axis=1)
x_test = x_test.drop(x_test_50.columns,axis=1)

In [62]:
n_feat = 101 - x_train_50.shape[1]

In [63]:
from sklearn.decomposition import KernelPCA

kernel = RationalQuadratic()
gpr = GaussianProcessRegressor(kernel = kernel, normalize_y = True)
gpr.fit(x_train, np.ravel(y_train))
#0.5174025667796436

FEATURE_SIZE = n_feat
pca = KernelPCA(n_components=FEATURE_SIZE, kernel = gpr.kernel_, random_state=0)
pca.fit(pd.concat([x_train, x_test], join = "inner"))
x_train_new = pd.DataFrame(pca.transform(x_train), columns = [str(i) for i in range(FEATURE_SIZE)], index = x_train.index)
x_test_new = pd.DataFrame(pca.transform(x_test), columns = [str(i) for i in range(FEATURE_SIZE)], index = x_test.index)

In [64]:
# from sklearn.datasets import make_regression
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.utils import plot_model
# from keras.layers import LeakyReLU
# from tensorflow.keras import layers
# from keras.layers import Dropout
# from matplotlib import pyplot
# from keras import regularizers
# import tensorflow as tf
# from numpy.random import seed
# import os
# seed(1)
# os.environ['PYTHONHASHSEED']=str(1)
# tf.random.set_seed(1)
# np.random.seed(1)
# n_inputs = x_train.shape[1]
# # define encoder
# input_layer = Input(shape=x_train.shape[1])
# #encoded = layers.Dense(1024, activation=LeakyReLU(alpha=0.01))(input_layer)
# #encoded = layers.Dense(100, activation='selu',kernel_regularizer=regularizers.L1(1))(input_layer)
# # encoded = layers.Dense(256, activation='selu')(encoded)
# encoded = Dropout(0.2,seed=1)(input_layer)
# bottleneck = layers.Dense(100-n_feat, activation='selu',kernel_regularizer=regularizers.l2(0.05))(encoded)
# # decoded = layers.Dense(512, activation='selu')(decoded)
# # decoded = layers.Dense(1024, activation='selu')(decoded)
# decoded = layers.Dense(x_train.shape[1], activation='selu')(bottleneck)
# autoencoder = Model(input_layer, decoded)
# autoencoder.compile(optimizer='SGD', loss='mean_absolute_error')

# history = autoencoder.fit(pd.concat([x_train, x_test], join = "inner"), pd.concat([x_train, x_test], join = "inner"),
#                 epochs=3000,
#                 batch_size=16,)

# # fit the autoencoder model to reconstruct input
# #history = model.fit(x_train, x_train, epochs=100, batch_size=16, verbose=2, validation_data=(x_test,x_test))
# # plot loss
# pyplot.plot(history.history['loss'], label='train')
# #pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()
# # # define an encoder model (without the decoder)
# encoder = Model(inputs=input_layer, outputs=bottleneck)
# encoder.compile(optimizer='SGD', loss='mean_absolute_error')

# # plot_model(encoder, 'encoder.png', show_shapes=True)
# # # save the encoder to file
# #encoder.save('./encoder/encoder400_epoch.h5')

# x_train_new = encoder.predict(x_train)
# x_test_new = encoder.predict(x_test)

In [65]:
x_train_new = pd.DataFrame(x_train_new, index = x_train.index)
x_test_new = pd.DataFrame(x_test_new, index = x_test.index)

# from sklearn.preprocessing import StandardScaler
# s = StandardScaler()
# s.fit(np.array(pd.concat([x_train_new, x_test_new], join = "inner")))
# x_train_new[[col for col in x_train_new.columns]] = s.transform(np.array(x_train_new))
# x_test_new[[col for col in x_train_new.columns]] = s.transform(np.array(x_test_new))

x_train_new = np.array(x_train_new)
x_test_new = np.array(x_test_new)

x_train = pd.concat([x_train_50,pd.DataFrame(x_train_new, index = x_train.index)],axis=1)
x_train.describe()
x_test = pd.concat([x_test_50,pd.DataFrame(x_test_new, index = x_test.index)],axis=1)

In [66]:
x_train.to_csv(preprocessed + 'X_train_preprocessed.csv',index_label="id")
x_test.to_csv(preprocessed + 'X_test_preprocessed.csv',index_label="id")
y_train.to_csv(preprocessed + 'y_train_preprocessed.csv',index_label="id")
y_test.to_csv(preprocessed + 'y_test_preprocessed.csv',index_label="id")

In [67]:
gpr.kernel_

RationalQuadratic(alpha=0.332, length_scale=1.27)